# Backtracking 回溯法

回溯法可以看作是蛮力法的优化版，因为它可以通过剪枝的策略来少走冤枉路。

具体做法：
1. 将问题的求解分为多个过程；
2. 每个阶段都面临一个岔路口，先随便选一条路走，当发现这条路走不通时（不符合期望解），就退回到上一个路口，另选一种走法。
3. 回溯非常适合用递归来实现。在实现过程中，剪枝操作是提高回溯效率的一种技巧。

## 1 - 0-1背包问题

### 1.1 - 简单版

有一个背包，其承重量是W kg。有n个物品，每个物品的重量为w[i]，且不可分割。求在不超过背包承重量的前提下，背包的最大重量。

In [1]:
def Backtrack(i, w, capacity):
    global cur_w, max_w
    if cur_w == capacity or i >= len(w):
        if cur_w > max_w:
            max_w = cur_w
    else:
        # 满足条件时：装入第i个物品，再装入第i+1个物品
        if cur_w + w[i] <= capacity:
            cur_w += w[i]
            Backtrack(i + 1, w, capacity)
            # 开始回溯
            cur_w -= w[i]
        # 不放入第i个物品，直接放入第i+1个物品
        Backtrack(i + 1, w, capacity)

In [2]:
w = [2, 2, 6, 5, 4]
cur_w, max_w = 0, 0
Backtrack(0, w, 10)
print(max_w)

10


### 1.2 - 进阶版
有一个背包，其承重量是W kg。有n个物品，每个物品的重量和价值分别为w[i]和v[i]，且不可分割。求在不超过背包承重量的前提下，背包中物品的最大价值。

**策略**：

- 使用0/1序列表示物品的放入情况。
- 将搜索看作一棵二叉树，二叉树的第i层代表第i个物品，左子树表示1（放入物品），右子树表示0（不放入物品）。如果背包剩余空间允许放入物品i，则扩展左子树；若不允许，则判断限界条。
- 当层数达到物品个数n时，停止扩展，开始回溯。（如何回溯？怎样得到就怎样恢复——放入背包的重量取出，对应价值减去）

约束条件：放入背包物品的总重量 <= 背包容量

限界条件：当前放入背包中物品的总价值 + 剩余物品的总价值 > 已知最有价值。这种情况可以继续往下搜索，否则停止搜索。

In [34]:
def Backtrack(i, w, v, capacity):
    global cur_v, max_v, cur_w, max_w, bag, best
    if i >= len(w):
        if cur_v > max_v:
            max_v = cur_v
            best = bag[:]
    else:
        if cur_w + w[i] <= capacity:
            cur_w += w[i]
            cur_v += v[i]
            bag[i] = 1
            Backtrack(i + 1, w, v, capacity)
            cur_w -= w[i]
            cur_v -= v[i]
        bag[i] = 0
        Backtrack(i + 1, w, v, capacity)

In [35]:
w = [2, 2, 6, 5, 4]
v = [6, 3, 5, 4, 6]
bag = [0 for i in range(len(w))]
best = [0 for i in range(len(w))]
cur_v, max_v, cur_w, max_w = 0, 0, 0, 0
Backtrack(0, w, v, 10)
print(max_v)
print(best)

15
[1, 1, 0, 0, 1]


## 2 - 正则表达式匹配

请实现一个函数用来匹配包括'.'和'\*'的正则表达式。模式中的字符'.'表示任意一个字符，而'\*'表示它前面的字符可以出现任意次（包含0次）。 在本题中，匹配是指字符串的所有字符匹配整个模式。例如，字符串"aaa"与模式"a.a"和"ab\*ac\*a"匹配，但是与"aa.a"和"ab\*a"均不匹配。

In [5]:
def Match(s, p):
    if len(s) == 0 and len(p) == 0:
        return True
    if len(s) > 0 and len(p) == 0:
        return False
    if len(s) == 0 and len(p) > 0:
        if len(p) > 1 and p[1] == '*':
            return Match(s, p[2:])
        return False
    if len(p) > 1 and p[1] == '*':
        if s[0] == p[0] or p[0] == '.':
            return Match(s[1:], p) or Match(s[1:], p[2:]) or Match(s, p[2:])
        return Match(s, p[2:])
    else:
        if s[0] == p[0] or p[0] == '.':
            return Match(s[1:], p[1:])
        return False

In [6]:
# test cases
s = 'aaa'
pattern1 = 'ab*ac*a'
pattern2 = 'aa.*'
print(Match(s, pattern1))
print(Match(s, pattern2))

True
True


## 3 - 矩阵中的路径

请设计一个函数，用来判断在一个矩阵中是否存在一条包含某字符串所有字符的路径。路径可以从矩阵中的任意一个格子开始，每一步可以在矩阵中向左，向右，向上，向下移动一个格子。如果一条路径经过了矩阵中的某一个格子，则该路径不能再进入该格子。 例如 [ a b c e ；s f c s； a d e e ]矩阵中包含一条字符串"bcced"的路径，但是矩阵中不包含"abcb"路径，因为字符串的第一个字符b占据了矩阵中的第一行第二个格子之后，路径不能再次进入该格子。

注：用字符串储存矩阵，以节省空间。

In [8]:
def HasPath(matrix, rows, cols, path):
    visited = [False] * len(matrix)
    
    def FindNext(i, j, k):
        if visited[i * cols + j]:
            return False
        if k == len(path):
            return True
        
        visited[i * cols + j] = True
        
        up, down, left, right = False, False, False, False
        if i > 0 and matrix[(i - 1) * cols + j] == path[k]:
            up = FindNext(i - 1, j, k + 1)
        if i < rows - 1 and matrix[(i + 1) * cols + j] == path[k]:
            down = FindNext(i + 1, j, k + 1)
        if j > 0 and matrix[i * cols + (j - 1)] == path[k]:
            left = FindNext(i, j - 1, k + 1)
        if j < cols - 1 and matrix[i * cols + (j + 1)] == path[k]:
            right = FindNext(i, j + 1, k + 1)
        
        res = (up or down or left or right)
        if not res:
            visited[i * cols + j] = False
        return res
    
    for i in range(rows):
        for j in range(cols):
            if matrix[i * cols + j] == path[0]:
                if FindNext(i, j, 1):
                    return True
    return False

In [10]:
# test case 1
matrix = 'abtgcfcsjdeh'
path = 'bfce'
res = HasPath(matrix, 3, 4, path)
print(res)

True


In [12]:
# test case 2
matrix = "ABCESFCSADEE"
path = "ABCB"
res = HasPath(matrix, 3, 4, path)
print(res)

False


## 4 - 二叉树中和为某一值的路径

输入一棵二叉树的根节点和一个整数，打印出二叉树中结点值的和为输入整数的所有路径。路径定义为从树的根结点开始往下一直到叶结点所经过的结点形成一条路径。(注意: 在返回值的list中，数组长度大的数组靠前)。

In [22]:
class TreeNode:
    def __init__(self, x, left, right):
        self.val = x
        self.left = left
        self.right = right

class Solution:
    # 返回二维列表，内部每个列表表示找到的路径
    def FindPath(self, root, expectNumber):
        # write code here
        res = []

        def main(node, expectNumber, path):
            path.append(node)
            is_leaf = node.left is None and node.right is None
            if is_leaf and node.val == expectNumber:
                one_path = [e.val for e in path]
                res.append(one_path)
            else:
                if node.left:
                    main(node.left, expectNumber - node.val, path)
                if node.right:
                    main(node.right, expectNumber - node.val, path)
            path.pop()

        main(root, expectNumber, [])

        return res

In [23]:
# test case
n4 = TreeNode(4, None, None)
n7 = TreeNode(7, None, None)
n5 = TreeNode(5, n4, n7)
n12 = TreeNode(12, None, None)
root = TreeNode(10, n5, n12)

S = Solution()
result = S.FindPath(root, 22)
print(result)

[[10, 5, 7], [10, 12]]


## 5 - 机器人的运动范围

In [24]:
class Solution:
    def movingCount(self, threshold, rows, cols):
        if threshold < 0 or rows <= 0 or cols <= 0:
            return False
        visited = [[False] * cols for _ in range(rows)]

        def Main(i, j):
            count = 0
            if self.Check(threshold, rows, cols, visited, i, j):
                visited[i][j] = True
                count = 1 + Main(i - 1, j) + Main(i + 1, j) + Main(i, j - 1) + Main(i, j + 1)
            return count

        return Main(0, 0)

    def Check(self, threshold, rows, cols, visited, i, j):
        if 0 <= i < rows and 0 <= j < cols and visited[i][j] is False\
                and self.GetDigitSum(i) + self.GetDigitSum(j) <= threshold:
            return True
        return False

    def GetDigitSum(self, n):
        sum = 0
        while n != 0:
            sum += n % 10
            n = n // 10
        return sum

In [25]:
S = Solution()
res = S.movingCount(5, 10, 10)
print(res)

21
